# Train

This notebook train a neural network. It dependes on the `train.hdf5` and `test.hdf5` files. So make sure you run the `rosbag_to_hdf5.ipynb` notebook first. 

## Network architecture

We use a convolutional neural network. The input is of size `(60, 80)`. We use a building block of 5x5 convolutional layer with padding, batch normalization, relu activation and then a max pooling layer. This is repeated until the feature maps reach size `(6, 8)`. For more details see the code in the `model.py` code module.

## Data augmentation

The images are augmented by random crops, brightness adapation and contrast changes. For details the the `data.py` module.


In [ ]:
from __future__ import division
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import h5py
import os
import six
from six.moves import range
import itertools
import matplotlib.pyplot as plt
import PIL
from tqdm import tqdm
from PIL import ImageOps
from PIL import ImageEnhance
import matplotlib.font_manager
from PIL import ImageDraw, ImageFont, ImageFilter
import tempfile
from datetime import datetime
import shutil

from deep_car.data import augment_img, augment_batch, crop_batch
from deep_car.model import Model

In [ ]:
data_dir = '../data'
model_dir = '../data/model'
model_name = 'steering_mixture_prob_exp'
tmp_dir = '../tmp'

os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
crop_size = (64, 48)

h5_train = h5py.File(os.path.join(data_dir, 'train.hdf5'))
h5_test = h5py.File(os.path.join(data_dir, 'test.hdf5'))

In [ ]:
print("{:25}| {:10}| {:30}".format("name", "dtype", "shape"))
print("-" * 40)
for name, dset in h5_train.items():
    print("{:25}| {:10}| {:30}".format(name, str(dset.dtype), str(dset.shape)))

In [ ]:
def data_generator(h5, batch_size=128, n_epoch=-1, shuffle=True, steering_distance_max=500):
    def get_steering(idx):
        idx = clip(idx)
        return np.array(steering[idx])

    def clip(x):
        return np.clip(x, 0, n-1)

    steering = np.array(h5['steering'])

    n = len(h5['image'])
    idx = np.arange(n)
    if n_epoch == -1:
        n_epoch = 1000000000

    for epoch in range(n_epoch):
        if shuffle:
            np.random.shuffle(idx)
        for b in range(0, n, batch_size):
            batch_idx = np.sort(idx[b:b+batch_size])            
            batch = {
                'image': h5['image'][batch_idx, :, :],
                'steering_abs': h5['steering'][batch_idx, :],
            }
            yield batch

In [ ]:
batch = next(data_generator(h5_train))
for name, arr in sorted(batch.items()):
    print("{:<17} | {:} ".format(name, arr.shape))

In [ ]:
batch['image'].min(), batch['image'].max(), 

### Benchmark the data iterator 

In [ ]:
%%timeit -n 1

for batch in data_generator(h5_test, n_epoch=1):
    pass

### Display the augmented images

In [ ]:
batch = next(data_generator(h5_train))
images = [PIL.Image.fromarray(x) for x in batch["image"]]
img = images[0]
fig, axes = plt.subplots(4, 12, figsize=(20, 5))

fig.suptitle("First row: Images from the dataset, Second row: Augmented images", fontsize=20)

images =  images[:len(axes[:1][0])]
for ax, img in zip(axes[:1].flat, images):
    ax.imshow(np.array(img), cmap='gray', vmin=0, vmax=255)
    ax.set_xticks([])
    ax.set_yticks([])

for ax, img in zip(axes[1:].flat, itertools.cycle(images)):
    ax.imshow(np.array(augment_img(img)), cmap='gray', vmin=0, vmax=255)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
def batch_to_numpy(batch):     
    x = 2. * batch['image'] / 255. - 1                                          
    steering_abs = batch['steering_abs']                                                                              
    return x[:, :, :, np.newaxis], steering_abs

In [ ]:
batch_aug = augment_batch(batch)
x_image, x_steering_abs = batch_to_numpy(batch_aug)

for name, arr in [
    ('data', x_image),
    ('steering_abs', x_steering_abs),
]:
    print("{:30} | {:20} | {:10} | {:10}".format(name, str(arr.shape), float(arr.min()), float(arr.max())))


In [ ]:
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

input_shape = [None, crop_size[1], crop_size[0], 1]
m = Model(input_shape)

init_op = tf.global_variables_initializer()
sess.run(init_op)

In [ ]:
history = {}
for name in ['steering_abs']:
    history[name] = []
    history["val_" + name] = []

In [ ]:
batch_size = 100
n_batches_per_epoch = len(h5_train['image']) // batch_size

tqdm_gen = tqdm(data_generator(h5_train, batch_size=batch_size, n_epoch=20))
running_loss = 'init'

saver = tf.train.Saver()
now = datetime.now()
save_dir = os.path.join(model_dir, model_name + "_" + now.isoformat())
os.makedirs(save_dir)

for i, batch in enumerate(tqdm_gen):
    x_image, x_steering_abs = batch_to_numpy(augment_batch(batch))
    steering_loss, _ = sess.run(
        [m.steering_abs_loss, m.opt_op], 
        feed_dict={
            m.image: x_image, 
            m.steering_abs_true: x_steering_abs,
            m.training: True,
    })
    
    history['steering_abs'].append(np.mean(steering_loss))
    batch_loss = np.mean(steering_loss)
    if running_loss == 'init':
        running_loss = batch_loss
    else:
        running_loss = 0.9*running_loss + 0.1*batch_loss
        
    if i % n_batches_per_epoch == 0:
        val_steering_abs = []
        for test_batch in data_generator(h5_test, batch_size=batch_size, n_epoch=1):
            x_image, x_steering_abs = batch_to_numpy(crop_batch(test_batch))
            steering_abs_loss = sess.run(
                [m.steering_abs_loss],
                feed_dict={
                    m.image: x_image, 
                    m.steering_abs_true: x_steering_abs,
                }
            )
            val_steering_abs.append(np.mean(steering_abs_loss))
            
        history['val_steering_abs'].append(np.mean(val_steering_abs))
            
    tqdm_gen.set_description('loss: {:.02f} - val_loss: {:.02f}'.format(running_loss, history['val_steering_abs'][-1]))

save_path = saver.save(sess, os.path.join(save_dir, model_name + ".ckpt"))
print("Saved model in: " + os.path.abspath(save_dir))